<a href="https://colab.research.google.com/github/guswns3396/PSYC-221-Project/blob/main/Segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/guswns3396/PSYC-221-Project.git

In [5]:
import os
from pathlib import Path

import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset

In [ ]:
!conda list

# packages in environment at C:\Users\yanghyun\Anaconda3\envs\psyc221:
#
# Name                    Version                   Build  Channel
abseil-cpp                20211102.0           hd77b12b_0  
aiobotocore               2.5.0           py311haa95532_0  
aiofiles                  22.1.0          py311haa95532_0  
aiohttp                   3.8.5           py311h2bbff1b_0  
aioitertools              0.7.1              pyhd3eb1b0_0  
aiosignal                 1.2.0              pyhd3eb1b0_0  
aiosqlite                 0.18.0          py311haa95532_0  
alabaster                 0.7.12             pyhd3eb1b0_0  
anaconda                  2023.09             py311_mkl_1  
anyio                     3.5.0           py311haa95532_0  
aom                       3.6.0                hd77b12b_0  
appdirs                   1.4.4              pyhd3eb1b0_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0          py311h2bbff1b_0  
arrow               

In [ ]:
path_train_data = Path('train')

In [ ]:
subjs = pd.Series(os.listdir(path_train_data))

In [ ]:
IMG_TYPES = {
    'bravo': 0,
    'flair': 1,
    't1_gd': 2,
    't1_pre': 3,
    'seg': 4
}

In [ ]:
data = np.array([])

In [ ]:
for subj in subjs:
    fnames = os.listdir(path_train_data / subj)
    imgs = np.array([])
    for img_typ in IMG_TYPES:
        nii_img = nib.load(path_train_data / subj / Path(img_typ + '.nii.gz'))
        np.append(imgs, nii_img.get_fdata())
    np.append(data, imgs)

KeyboardInterrupt: 